In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S6'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #5 as in paper, not 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 19998
Total Tokens Found = 292928
Total Tokens Output = 292542
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 19998
Total Tokens= 292542
Unique Tokens= 35953
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 19998
Sorted Token Size = 35953
Clean Token Size = 12890
*Stop Replacements here
References Processed =  19998
Total Replacement Pairs = 3806
Tokens Read = 292542
Tokens Changed =  4447
References Changed = 4017


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.1,beta=35,sigma=403,epsilon=15.1,epsilonIterate=0.0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 15.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 15949
NewMatrix Iterateblocks Total Time = 21.464176503941417


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 15.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 14369
NewMatrix Iterateblocks Total Time = 18.094705058261752


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 15.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9006
NewMatrix Iterateblocks Total Time = 12.189565903507173


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 15.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 3729
NewMatrix Iterateblocks Total Time = 4.452491511590779


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 15.1
Running NewMatrix Jar!
 Single-Refe

0.9026049

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [15]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [16]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


0.603939396297269

In [17]:
optimizer=None

In [18]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [19]:
from bayes_opt import SequentialDomainReductionTransformer

In [20]:
bounds_transformer = SequentialDomainReductionTransformer()

In [21]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [22]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(35), 
#             "sigma_sp": inv_softplus(403),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(15.1)},
#     lazy=True,
# )

In [23]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  0.4034   |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  0.8666   |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        | -3.006    |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        | -0.1      |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        | -1.823    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  1.93     |  20.72    |  149.6    | -3.1      |  0.9905   |  434.2    |
Mu:0.72|Mu+:0.04|Beta:25.00|Sigma:437.00|Epsilon:152.90|EpIter:0.00
|  40       |  1.855    |  25.34    |  152.9    | -3.087    |  0.9669   |  437.8    |
Mu:0.72|Mu+:0.04|Beta:18.00|Sigma:431.00|Epsilon:146.55|EpIter:0.00
|  41       |  1.919    |  18.29    |  146.6    | -3.088    |  0.9693   |  431.0    |
Mu:0.73|Mu+:0.04|Beta:16.00|Sigma:437.00|Epsilon:146.85|EpIter:0.00
|  42       |  1.911    |  16.97    |  146.9    | -3.089    |  0.9714   |  437.1    |
Mu:0.73|Mu+:0.04|Beta:22.00|Sigma:431.00|Epsilon:151.99|EpIter:0.00
|  43       |  1.876    |  22.82    |  152.0    | -3.109    |  0.9733   |  431.6    |
Mu:0.73|Mu+:0.04|Beta:23.00|Sigma:436.00|Epsilon:147.37|EpIter:0.00
|  44       |  1.86     |  23.75    |  147.4    | -3.091    |  0.975    |  436.6    |
Mu:0.73|Mu+:0.04|Beta:17.00|Sigma:432.00|Epsilon:151.53|EpIter:0.00
|  45       |  1.927    |  17.99    |  151.5    | -3.107    |  1.004    |  4

In [24]:
optimizer.max

{'target': 1.9418033804584731,
 'params': {'beta_sp': 20.905391979583307,
  'epsilon_sp': 153.24182604121094,
  'muIncr_logit': -3.114114851122126,
  'mu_logit': 1.0164819714982578,
  'sigma_sp': 436.97841814418996}}

In [25]:
inv_logit(optimizer.max["target"])

0.87455013

In [24]:
params=optimizer.max["params"]

In [25]:
softplus(params["beta_sp"])

34.56027510764839

In [26]:
softplus(params["sigma_sp"])

406.4487059734664

In [27]:
inv_logit(params["mu_logit"])

0.5264055778635047

In [28]:
inv_logit(params["muIncr_logit"])

0.07890602612671446

In [29]:
softplus(params["epsilon_sp"])

13.862860100699521

In [31]:
entropy_F1(mu=0.5264055778635047,
           muIncr=0.07890602612671446,
           beta=34,
           sigma=406,
           epsilon=13.862860100699521,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5264055778635047
Starting Iteration epsilon= 13.862860100699521
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 15760
NewMatrix Iterateblocks Total Time = 19.58311090618372


>>>>>>>>>>>>
Starting Iteration mu= 0.6053116039902192
Starting Iteration epsilon= 13.862860100699521
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 13181
NewMatrix Iterateblocks Total Time = 17.280983736738563


>>>>>>>>>>>>
Starting Iteration mu= 0.6842176301169337
Starting Iteration epsilon= 13.862860100699521
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9030
NewMatrix Iterateblocks Total Time = 11.896439941599965


>>>>>>>>>>>>
Starting Iteration mu= 0.7631236562436482
Starting Iteration epsilon= 13.862860100699521
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 4957
NewMatrix Iterateblocks Total Time = 6.51996822655201


>>>>>>>>>>>>
Starting It

0.91384856